In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
sys.path.append(os.getcwd() + '/..')
import src

# Experiment setup
* Here we basically just load the game and check that it works

In [3]:
from __future__ import print_function 
import matplotlib.pyplot as plt
import numpy as np
import gym
from tqdm import tqdm


%matplotlib inline
%env THEANO_FLAGS="floatX=float32"

env: THEANO_FLAGS="floatX=float32"


In [4]:
GAME = "FrontoPolarStocks-v0"

N_AGENTS = 1
SEQ_LENGTH = 10

In [5]:
env = gym.make(GAME)
env.reset()

action_shape = (env.action_space.num_discrete_space,)
action_emb_shape = (*action_shape, 3)
state_shape = env.observation_space.shape

state, _, _, _ = env.step([0] * action_shape[0])

action_names = np.array(["sell", "pass", "buy"]) #i guess so... i may be wrong

print(state)

[2017-05-21 21:30:23,663] Making new env: FrontoPolarStocks-v0


[ 452.500023   54.699997   37.830002   42.919998   23.190001   34.119999
   24.23       51.93       24.200001   22.799999   26.200001   34.82
    8.94       14.55    ]


# Basic agent setup
Here we define a simple agent that maps game images into Qvalues using shallow neural network.


In [6]:
import theano
from theano import tensor as T
import lasagne

theano.config.exception_verbosity = 'high'
theano.config.optimizer = 'fast_compile'

In [7]:
from agent.agent import build_agent

In [8]:
agent, action_layer, V_layer = build_agent(action_emb_shape, state_shape)

In [9]:
agent.action_layers[0].output_shape

(None, 14)

In [10]:
#Since it's a single lasagne network, one can get it's weights, output, etc
weights = lasagne.layers.get_all_params((action_layer,V_layer),trainable=True)

# Create and manage a pool of atari sessions to play with

* To make training more stable, we shall have an entire batch of game sessions each happening independent of others
* Why several parallel agents help training: http://arxiv.org/pdf/1602.01783v1.pdf
* Alternative approach: store more sessions: https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf

In [11]:
from agentnet.experiments.openai_gym.pool import EnvPool

pool = EnvPool(agent, GAME, N_AGENTS, max_size=10000)

[2017-05-21 21:30:25,826] Making new env: FrontoPolarStocks-v0


In [12]:
%%time
#interact for 7 ticks
_,action_log,reward_log,_,_,_  = pool.interact(7)


print(action_names[action_log])
print(reward_log)

[[['buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy'
   'buy' 'buy']
  ['buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy'
   'buy' 'buy']
  ['buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy'
   'buy' 'buy']
  ['buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy'
   'buy' 'buy']
  ['buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy'
   'buy' 'buy']
  ['buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy' 'buy'
   'buy' 'buy']
  ['pass' 'buy' 'pass' 'pass' 'pass' 'pass' 'pass' 'buy' 'buy' 'pass' 'buy'
   'pass' 'pass' 'buy']]]
[[  3.36947898 -18.36905801 -28.06317896 -14.633176    61.47429002
  -22.61297396   0.        ]]
CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 18.3 ms


In [13]:
#load first sessions (this function calls interact and remembers sessions)
pool.update(SEQ_LENGTH)

# a2c loss

Here we define obective function for actor-critic (one-step) RL.

* We regularize policy with expected inverse action probabilities (discouraging very small probas) to make objective numerically stable


In [14]:
#get agent's Qvalues obtained via experience replay
replay = pool.experience_replay.sample_session_batch(100, replace=True)

_,_,_,_,(policy_seq,V_seq) = agent.get_sessions(
    replay,
    session_length=SEQ_LENGTH,
    experience_replay=True,
)

In [15]:
#get reference Qvalues according to Qlearning algorithm
from agentnet.learning import a2c


#Train via actor-critic (see here - https://www.youtube.com/watch?v=KHZVXao4qXs)

### COMMENTED ASSERT ACTION.DIM==2 LINE IN THIS METHOD
elwise_mse_loss = a2c.get_elementwise_objective(policy_seq,V_seq[:,:,0],
                                                       replay.actions[0],
                                                       replay.rewards,
                                                       replay.is_alive,
                                                       gamma_or_gammas=0.99)

#compute mean over "alive" fragments
loss = elwise_mse_loss.sum() / replay.is_alive.sum()

In [16]:
reg = T.mean((1./policy_seq).sum(axis=-1))
loss += 0.001*reg

In [17]:
# Compute weight updates
updates = lasagne.updates.rmsprop(loss, weights, learning_rate=0.001)

In [18]:
#compile train function

train_step = theano.function([],loss,updates=updates)

# Demo run

In [19]:
#for MountainCar-v0 evaluation session is cropped to 200 ticks
untrained_reward = pool.evaluate(save_path="./records",record_video=True)

[2017-05-21 21:30:29,364] Making new env: FrontoPolarStocks-v0
[2017-05-21 21:30:29,595] Clearing 4 monitor files from previous run (because force=True was provided)
[2017-05-21 21:30:29,599] Starting new video recorder writing to /home/manatee/Desktop/inn.prac/Algorithms/src/records/openaigym.video.0.786.video000000.mp4
[2017-05-21 21:30:36,741] Finished writing results. You can upload them to the scoreboard via gym.upload('/home/manatee/Desktop/inn.prac/Algorithms/src/records')


Episode finished after 400 timesteps with reward=104771.62900974997


In [20]:
from IPython.display import HTML

video_path="records/openaigym.video.0.15895.video000000.mp4"

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(video_path))

# Vime

In [21]:
from agent.bnn import BNN
bnn = BNN(state_shape, action_shape, action_emb_shape, replay)

(None, 14, 3)
(None, 56)


# Training loop

In [22]:
#starting epoch
epoch_counter = 1

#full game rewards
rewards = {epoch_counter: untrained_reward}

In [23]:
#pre-fill pool
for i in tqdm(range(1000)):
    pool.update(SEQ_LENGTH, append=True, preprocess=bnn.add_vime_reward)

  0%|          | 0/1000 [00:00<?, ?it/s]


ValueError: Shape mismatch: x has 17 cols (and 10 rows) but y has 56 rows (and 50 cols)
Apply node that caused the error: Dot22(Join.0, Elemwise{add,no_inplace}.0)
Toposort index: 160
Inputs types: [TensorType(float32, matrix), TensorType(float32, matrix)]
Inputs shapes: [(10, 17), (56, 50)]
Inputs strides: [(68, 4), (200, 4)]
Inputs values: ['not shown', 'not shown']
Inputs type_num: [11, 11]
Outputs clients: [[Elemwise{add,no_inplace}(Dot22.0, InplaceDimShuffle{x,0}.0)]]

Debugprint of the apply node: 
Dot22 [id A] <TensorType(float32, matrix)> ''   
 |Join [id B] <TensorType(float32, matrix)> ''   
 | |TensorConstant{1} [id C] <TensorType(int8, scalar)>
 | |Reshape{2} [id D] <TensorType(float32, matrix)> ''   
 | | |AdvancedSubtensor1 [id E] <TensorType(float32, matrix)> ''   
 | | | |Elemwise{add,no_inplace} [id F] <TensorType(float32, matrix)> ''   
 | | | | |bnn.mu_copy [id G] <TensorType(float32, matrix)>
 | | | | |Elemwise{mul,no_inplace} [id H] <TensorType(float32, matrix)> ''

In [ ]:
#pre-train BNN (mitigate training lag on first iterations where BNN is stupid)
for i in tqdm(range(1000)):
    bnn.train_step(*bnn.sample_from_pool())

In [ ]:
#the loop may take eons to finish.
#consider interrupting early.
loss = 0
for i in tqdm(range(1000)):    
    
    
    #train
    for i in range(10):
        pool.update(SEQ_LENGTH,append=True,preprocess=bnn.add_vime_reward)

    for i in range(10):
        loss = loss*0.99 + train_step()*0.01
    
    for i in range(10):
        bnn.train_step(*bnn.sample_from_pool())

    if epoch_counter%100==0:
        #average reward per game tick in current experience replay pool
        pool_mean_reward = np.average(pool.experience_replay.rewards.get_value()[:,:-1],
                                      weights=1+pool.experience_replay.is_alive.get_value()[:,:-1])
        pool_size = pool.experience_replay.rewards.get_value().shape[0]
        print("iter=%i\treward/step=%.5f\tpool_size=%i\tvime ma=%.5f"%(epoch_counter,
                                                         pool_mean_reward,
                                                         pool_size,
                                                         bnn.vime_reward_ma))
        

    ##record current learning progress and show learning curves
    if epoch_counter%500 ==0:
        n_games = 10
        rewards[epoch_counter] = pool.evaluate( record_video=False,n_games=n_games,verbose=False)
        print("Current score(mean over %i) = %.3f"%(n_games,np.mean(rewards[epoch_counter])))
    
    
    epoch_counter  +=1

    
# Time to drink some coffee!

In [ ]:
iters, session_rewards=zip(*sorted(rewards.items(),key=lambda pr:pr[0]))

In [ ]:
plt.plot(iters,list(map(np.mean, session_rewards)))
plt.title("Training progress")
plt.xlabel("Epoch counter")
plt.ylabel("Mean Income")
plt.show()

In [ ]:
_,_,_,_,(pool_policy,pool_V) = agent.get_sessions(
    pool.experience_replay,
    session_length=SEQ_LENGTH,
    experience_replay=True,)

states = pool.experience_replay.observations[0].get_value().reshape((-1, 5)).T[-2:]
values = pool_V.ravel().eval()
optimal_actid = pool_policy.argmax(-1).ravel().eval()

In [ ]:
plt.scatter(
    *states,
    c=values,
    alpha = 0.1)
plt.title("predicted state values")
plt.xlabel("previous")
plt.ylabel("current")
plt.show()

In [ ]:
obs_x, obs_y = states

for i in range(3):
    sel = optimal_actid==i
    plt.scatter(obs_x[sel],obs_y[sel],
                c=['red','blue','green'][i],
                alpha = 0.1,label=action_names[i])
    
plt.title("most likely action id")
plt.xlabel("previous")
plt.ylabel("current")
plt.legend(loc='best')
plt.show()

In [ ]:
#for MountainCar-v0 evaluation session is cropped to 200 ticks
untrained_reward = pool.evaluate(save_path="./records",record_video=True)